### Fonctions utilitaires

Compléter en ajoutant les fonctions de padding et de découpage en bloc.

In [2]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.array(toLoad)


def psnr(original, compressed):
    mse = np.mean((original.astype(int) - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


### Utilitaires pour le calcul des palettes et des patchs

Fonctions auxiliaires pour manipuler les palettes, les pixels et les blocs. 

In [3]:
def load(filename):
    toLoad= Image.open(filename)
    return np.array(toLoad)

def save(matPix, filename):
    Image.fromarray(matPix).save(filename)

def padding(filename):
    mat = load(filename)

    ligne_mat = mat.shape[0]
    colonne_mat = mat.shape[1]

    #Obtenir le nombre de ligne et de colonne à rajouter.
    ligne_mat_manquante = (4 - ligne_mat % 4) % 4  
    colonne_mat_manquante = (4 - colonne_mat % 4) % 4

    #Création d'une matrice vide de la taille souhaitée. 
    mat_padding = np.zeros((ligne_mat + ligne_mat_manquante, colonne_mat + colonne_mat_manquante, 3),dtype = mat.dtype)
    mat_padding[:ligne_mat, :colonne_mat] = mat

    return mat_padding 

def decoupe_matrice(mat):
    ligne_mat = mat.shape[0]
    colonne_mat = mat.shape[1]

    #Création d'une liste qui va contenir tout les carre de 4x4 pixel en RGB.
    global Liste_des_blocs
    Liste_des_blocs=[]
    for ligne in range(0, ligne_mat, 4):
        for colonne in range(0, colonne_mat, 4):
            bloc = mat[ligne:ligne+4, colonne:colonne+4]
            Liste_des_blocs.append(bloc)

    return Liste_des_blocs

def non_padding(filename, ligne_de_base, colonne_de_base):
    #Padding
    mat_padding = padding(filename)
    #Découpe de la matrice
    Liste_des_blocs = decoupe_matrice(mat_padding)

    #Dimensions de la matrice recollée.
    ligne_mat = mat_padding.shape[0]
    colonne_mat = mat_padding.shape[1]

    #Création de la matrice recollée.
    mat_recoller = np.zeros((ligne_mat, colonne_mat, 3), dtype=mat_padding.dtype)
    for i in range(len(Liste_des_blocs)):
        ligne_idx = (i // (colonne_de_base // 4)) * 4
        col_idx = (i % (colonne_de_base // 4)) * 4
        mat_recoller[ligne_idx:ligne_idx+4, col_idx:col_idx+4] = Liste_des_blocs[i]

    #Retour à la matrice initiale.
    mat_de_base = mat_recoller[:ligne_de_base, :colonne_de_base]
    
    return mat_de_base

In [4]:
def tronque(nombre, p):
    #On met le nombres n en binaire.
    nombre_binaire = []
    while nombre != 0:
        nombre_binaire.append(nombre % 2)
        nombre = nombre // 2
    nombre_binaire.reverse()

    #On enlève les p bits de poids faible.
    nombre_binaire = nombre_binaire[:-p]
    
    #On remet le nombre en bianire.
    nombre_decimal = 0
    for i in range(len(nombre_binaire)):
        nombre_decimal += nombre_binaire[i] * (2 ** (len(nombre_binaire) - 1 - i))

    return nombre_decimal

def palette(a, b):
    #On crée notre palette.
    palette = np.zeros((4, 3), dtype=int)
    palette[0] = a
    palette[1] = np.round(2 * np.array(a) / 3 + np.array(b) / 3)
    palette[2] = np.round(np.array(a) / 3 + 2 * np.array(b) / 3)
    palette[3] = b
    
    return palette

def couleur_la_plus_proche(palette, pixel):
    #Initialise l'indice de la couleur la plus proche avec la première couleur de la palette.
    indice_couleur_plus_proche = 0
    distance_euclidienne_la_plus_petite = np.linalg.norm(palette[0].astype(int) - pixel)
    
    #On test avec les autre couleur de la palette.
    for indice in range (len(palette)):
        distance_euclidienne = np.linalg.norm(palette[indice].astype(int) - pixel)
        if distance_euclidienne < distance_euclidienne_la_plus_petite:
            indice_couleur_plus_proche = indice

    return indice_couleur_plus_proche

### Les méthodes de choix des couleurs de la palette


On peut essayer plusieurs stratégies pour calculer les deux couleurs a et b:
* Calculer la couleur minimale et maximale
* Prendre des couleurs moyennes (calculer moyenne et variance pour trouver les meilleurs valeurs) 
* Calculer 10 valeurs au hasard et prendre la meilleure
* Partir d'un choix et perturber pour améliorer le résultat: on teste la meilleure modification de 16 d'un des canaux d'une des couleurs

In [5]:
def creer_couleurs_min_max(bloc_image):
    #Initialise nos canaux RGB minimal et maximal.
    rouge_minimal, rouge_maximal = 255, 0
    vert_minimal, vert_maximal = 255, 0
    bleu_minimal, bleu_maximal = 255, 0

    #On modifie nos canaux RGB minimal et maximal.
    for ligne in bloc_image:
        for colonne in ligne:
            rouge_minimal, rouge_maximal = min(rouge_minimal, colonne[0]), max(rouge_maximal, colonne[0]) 
            vert_minimal, vert_maximal = min(vert_minimal, colonne[1]), max(vert_maximal, colonne[1])    
            bleu_minimal, bleu_maximal = min(bleu_minimal, colonne[2]), max(bleu_maximal, colonne[2]) 
    
    #On met nos couleur au bon format.
    couleur_minimale = ((tronque(rouge_minimal,3)), (tronque(vert_minimal,2)), (tronque(bleu_minimal,3)))
    couleur_maximale = ((tronque(rouge_maximal,3)), (tronque(vert_maximal,2)), (tronque(bleu_maximal,3)))

    return couleur_minimale, couleur_maximale

In [6]:

def creer_couleurs_moyenne_ecart_type(bloc):
    bloc = np.array(bloc)
    moyenne_rouge = np.mean(bloc[:, :, 0])  #Moyenne des composantes rouges de tous les pixels
    ecart_type_rouge = np.std(bloc[:, :, 0])  #Écart-type des composantes rouges de tous les pixels
    moyenne_vert = np.mean(bloc[:, :, 1])  
    ecart_type_vert = np.std(bloc[:, :, 1])
    moyenne_bleu = np.mean(bloc[:, :, 2])  
    ecart_type_bleu = np.std(bloc[:, :, 2])  

    rouge_min, rouge_max = moyenne_rouge - ecart_type_rouge, moyenne_rouge + ecart_type_rouge
    vert_min, vert_max = moyenne_vert - ecart_type_vert, moyenne_vert + ecart_type_vert
    bleu_min, bleu_max = moyenne_bleu - ecart_type_bleu, moyenne_bleu + ecart_type_bleu

    couleur_min = (int(tronque(rouge_min,3)), int(tronque(vert_min,2)), int(tronque(bleu_min,3)))
    couleur_max = (int(tronque(rouge_max,3)), int(tronque(vert_max,2)), int(tronque(bleu_max,3)))

    return couleur_min, couleur_max

### Encodage d'un bloc

Écrire ici le code qui permet d'encoder un bloc et une palette en un entier.

In [7]:
def patch(tableaux_des_indices, a, b, i):
    patch = ""
    #On met en binaire l'indice de la couleur la plus proche de la palette par rapport a tout les pixels.
    for indices in tableaux_des_indices:
        pixel = Liste_des_blocs[i][indices[0], indices[1]] #Pixel du block "i" de 4x4 pixel
        indice_couleur_plus_proche = couleur_la_plus_proche(palette(a, b), pixel)
        indice_couleur_plus_proche_binaire = bin(indice_couleur_plus_proche)[2:].zfill(2)
        patch += str(indice_couleur_plus_proche_binaire)

    #On ajoute nos couleur.
    patch += str(bin(b[2]))[2:].zfill(5) + str(bin(b[1]))[2:].zfill(6) + str(bin(b[0]))[2:].zfill(5) \
        + str(bin(a[2]))[2:].zfill(5) + str(bin(a[1]))[2:].zfill(6) + str(bin(a[0]))[2:].zfill(5)
    
    return patch

### Écriture et lecture dans un fichier

In [8]:
def informations_image(type_fichier, hauteur, largeur):
    with open("informations_image.txt", "w") as f:
        f.write(type_fichier + "\n")
        f.write(str(hauteur) + " " + str(largeur) + "\n")

def ecrire_patches(tableaux_des_indices):
    with open("informations_image.txt", "a") as f:
        #Boucle pour chaque bloc
        for i in range(len(Liste_des_blocs)):
            a, b = creer_couleurs_min_max(Liste_des_blocs[i])
            #Transforme notre patch binaire en decimal.
            patch_i = str(int(patch(tableaux_des_indices, a, b, i), 2))
            f.write(patch_i + "\n")

In [9]:
def Liste_des_patch(nom_du_ficher):
    with open(nom_du_ficher, "r") as fichier:
        Liste_des_patch  = []
        compteur = 0
        for ligne in fichier:
            if compteur < 2:
                compteur += 1
            else:
                Liste_des_patch.append(int(ligne.strip()))
                
    return Liste_des_patch

In [36]:
#Test de la fonction "patch"

tableaux_des_indices = [[3,3],[3,2],[3,1],[3,0],[2,3],[2,2],[2,1],[2,0],[1,3],[1,2],[1,1],[1,0],[0,3],[0,2],[0,1],[0,0]]
a, b = creer_couleurs_moyenne_ecart_type(Liste_des_blocs[0])
patch_0 = patch(tableaux_des_indices, a, b, 0)
print(patch_0)

def patch_binaire(patch):
    patch_entier = int(patch)
    patch_binaire = bin(patch_entier)[2:].zfill(64)  # Convertir en binaire et remplir avec des zéros à gauche pour une longueur de 64 bits
    print(patch_binaire)

# Supposons que patch_0 est défini quelque part
patch_0 = 296423851  # Par exemple

patch_binaire(patch_0)



0000000000000000000000000000000000010001101010110001000110101011
0000000000000000000000000000000000010001101010110001000110101011


### Fonctions de test

Écrire ici tous vos tests. Chaque fonction écrite doit être testée.

In [13]:
#Test de la fonction "padding"

mat_padding = padding("proc.jpg")
save(mat_padding, "proc_padding.jpg")

In [14]:
#Test de la fonction "decoupe_matrice"

decoupe_matrice(mat_padding)
pixel = Liste_des_blocs[0]
print(pixel)

[[[94 54 18]
  [94 54 18]
  [94 54 18]
  [94 54 18]]

 [[94 54 18]
  [94 54 18]
  [94 54 18]
  [94 54 18]]

 [[94 54 18]
  [94 54 18]
  [94 54 18]
  [94 54 18]]

 [[95 55 19]
  [95 55 19]
  [95 55 19]
  [95 55 19]]]


In [15]:
#Test de la fonction "non_padding"

mat_de_base = non_padding("proc.jpg",1537,2048)
save(mat_de_base, "proc_non_padding.jpg")

In [16]:
#Test de la fonction "tronque"

n = 255
p = 3   
n_tronque = tronque(n, p)
print("Nouvelle valeur", n_tronque)

Nouvelle valeur 31


In [17]:
#Test de la fonction "palette"

a = [tronque(90,3),tronque(90,2),tronque(90,3)]
b = [tronque(180,3),tronque(180,2),tronque(180,3)]
c = palette(a, b)
print(c)

[[11 22 11]
 [15 30 15]
 [18 37 18]
 [22 45 22]]


In [18]:
#Test de la fonction "couleur_la_plus_proche"

pixel = [100, 100, 100]
print(couleur_la_plus_proche(c, pixel))

3


In [19]:
#Test de la fonction "creer_couleurs_min_max"

couleur_min, couleur_max = creer_couleurs_min_max(Liste_des_blocs[0])
print(couleur_min, couleur_max)

(11, 13, 2) (11, 13, 2)


In [20]:
#Test de la fonction "creer_couleurs_moyenne_ecart_type"

couleur_min, couleur_max = creer_couleurs_moyenne_ecart_type(Liste_des_blocs[0])
print(couleur_min, couleur_max)

(11, 13, 2) (11, 13, 2)


In [21]:
#Test de la fonction "patch"

tableaux_des_indices = [[3,3],[3,2],[3,1],[3,0],[2,3],[2,2],[2,1],[2,0],[1,3],[1,2],[1,1],[1,0],[0,3],[0,2],[0,1],[0,0]]
a, b = creer_couleurs_moyenne_ecart_type(Liste_des_blocs[0])
patch_0 = patch(tableaux_des_indices, a, b, 0)
print(patch_0)

0000000000000000000000000000000000010001101010110001000110101011


In [ ]:
#Test de la fonction "informations_image"

informations_image("BC1", 1537, 2048)

In [ ]:
#Test de la fonction "ecrire_patches"

tableaux_des_indices = [[3,3],[3,2],[3,1],[3,0],[2,3],[2,2],[2,1],[2,0],[1,3],[1,2],[1,1],[1,0],[0,3],[0,2],[0,1],[0,0]]
ecrire_patches(tableaux_des_indices)

In [ ]:
#Test de la fonction "Liste_des_patch"

Liste_des_patch("informations_image.txt")

[296423851,
 296423851,
 298521035,
 298521035,
 298521035,
 298521035,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 18446744069715268075,
 300683756,
 300618219,
 300618219,
 18446744069715202507,
 298521035,
 298521035,
 298521035,
 298521035,
 18446744069983640011,
 18446744070122054123,
 18446744070394753611,
 18446744070667387499,
 18446744070805801676,
 18446744070944283404,
 18446744071080665901,
 18446744071219080046,
 18446744071357559726,
 18446744071493876687,
 18446744071630191599,
 18446744071632356368,
 2217772080,
 18446744071768738896,
 2356251761,
 18446744071772933233,
 18446744071907218577,
 2492634258,
 18446744072043533458,
 2628951218,
 2628951218,
 18446744072043533490,
 18446744072045630642,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631113939,
 2631113939,
 2767430899,
 27674

In [ ]:
a = [[[94, 54, 18], [94, 54, 18], [94, 54, 18], [94, 54, 18]],
     [[94, 54, 18], [94, 54, 18], [94, 54, 18], [94, 54, 18]],
     [[94, 54, 18], [94, 54, 18], [94, 54, 18], [94, 54, 18]],
     [[95, 55, 19], [95, 55, 19], [95, 55, 19], [95, 55, 19]]]